# Humanitarian Facility Location Optimisation Tutorial

---

## Overview

This tutorial walks you through the type of facility location optimisation problem that is faced by groups delivering aid to people in disaster zones. Whether the aid is medica, food assistance, shelter, or anything else, an optimal set of facility locations is imperitive in order to maximise the impact of the group's financial and human resources.

We will discuss some methods to describe such problems mathematically, apply genetic algorithms to find the optimal set of locations, and interact with maps of real places by importing data from OpenStreetMap.

The tutorial is not just a bunch of working code with some explanations of how it works sprinkled in - it's interactive, so you will be prompted to write code throughout to carry out specific tasks and solve the optimisation problems. When this is the case, a message with be outputted by the cell to tell you what parts of your code are correct and which parts need to be corrected.

## Setup

Run the following cell to setup the neccessary libraries for this tutorial.

In [ ]:
%%capture
!pip install networkx
!pip install deap
import networkx as nx
import numpy as np
import deap
import matplotlib.pyplot as plt
from itertools import combinations

Also run this cell to setup the continuous feedback for this tutorial:

In [ ]:
# @title
from re import S
from IPython.core.magic import register_cell_magic
from deap import base, creator, tools, algorithms
import random
from inspect import signature

# Reference solution for comparison
def reference_calculate_max_distance_to_facility(G, hotspot_nodes, facility_nodes):
    distances = {hotspot: float('inf') for hotspot in hotspot_nodes}

    for facility in facility_nodes:
        facility_distances = nx.single_source_shortest_path_length(G, facility)
        for hotspot in hotspot_nodes:
            if hotspot in facility_distances:
                distances[hotspot] = min(distances[hotspot], facility_distances[hotspot])

    return max(distances.values())

def reference_find_optimal_facilities(G, hotspot_nodes, num_facilities):
    best_facilities = None
    min_max_distance = float('inf')

    for facilities in combinations(G.nodes(), num_facilities):
        max_distance = reference_calculate_max_distance_to_facility(G, hotspot_nodes, facilities)
        if max_distance < min_max_distance:
            min_max_distance = max_distance
            best_facilities = facilities

    return best_facilities, min_max_distance


def perform_max_dist_check():
    if 'calculate_max_distance_to_facility' in globals():
        ref_max_dist = []
        student_max_dist = []
        for facilities in combinations(G.nodes(), 2):
            ref_max_dist.append(reference_calculate_max_distance_to_facility(G, hotspot_nodes, facilities))
            student_max_dist.append(calculate_max_distance_to_facility(G, hotspot_nodes, facilities))
        if (ref_max_dist == student_max_dist):
            print("✅ calculate_max_distance_to_facility is outputting the correct values")
        else:
            print("❌ calculate_max_distance_to_facility is not outputting the correct values")
    else:
        print("❌ calculate_max_distance_to_facility is not defined")


# Function to perform all checks
def perform_checks_brute_force():
    ref_facilities, ref_max_distance = reference_find_optimal_facilities(G, hotspot_nodes, num_facilities)
    # Check for the correct type and values of objects
    if 'optimal_facilities' in globals():
        if ref_facilities==optimal_facilities:
            print("✅ optimal_facilities is a tuple of tuples")
        else:
            print("❌ optimal_facilities is not a tuple of tuples")
    else:
        print("❌ optimal_facilities is not defined")

    if 'optimal_max_distance' in globals():
        # Validate with reference value if available
        if np.isclose(optimal_max_distance, ref_max_distance, atol=1e-5):
            print("✅ optimal_max_distance is correct.")
        else:
            print(f"❌ optimal_max_distance is not correct.")
    else:
        print("❌ optimal_max_distance is not defined")

# Define the cell magic function
@register_cell_magic
def check_brute_force(line, cell):
    # Execute the student's code
    exec(cell, globals())
    # Run checks
    perform_checks_brute_force()

@register_cell_magic
def check_calculate_max_distance_to_facility(line, cell):
    # Execute the student's code
    exec(cell, globals())
    # Run checks
    perform_max_dist_check()

# Evaluation function for the genetic algorithm
def reference_evaluate(individual):
    facility_nodes = list(set(individual))
    max_distance = calculate_max_distance_to_facility(G, hotspot_nodes, facility_nodes) #We haven't applied the minus sign yet, that will come later
    return (max_distance,)

# Mutation function - it just replaces nodes with random nodes from the network with probability indpb
def reference_mutate(individual, num_nodes, hotspot_nodes, indpb):
    if random.random() < indpb:
        idx = random.randint(0, len(individual) - 1)
        new_node = random.choice(list(G.nodes()))
        while new_node in hotspot_nodes or new_node in individual:
            new_node = random.choice(list(G.nodes()))
        individual[idx] = new_node
    return individual,

def reference_genetic_algorithm(G, hotspot_nodes, num_facilities, generations=50, population_size=200):
    num_nodes = len(G.nodes())

    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("attr_node", lambda: random.choice([node for node in G.nodes() if node not in hotspot_nodes]))
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_node, n=num_facilities)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", evaluate)
    toolbox.register("mate", Mate)
    toolbox.register("mutate", mutate, num_nodes=num_nodes, hotspot_nodes=hotspot_nodes, indpb=0.1)
    toolbox.register("select", tools.selTournament, tournsize=10)

    population = toolbox.population(n=population_size)

    return toolbox, generations, population_size, 10

# Function to perform all checks
def perform_checks_genetic_algorithm_complete(G, hotspot_nodes, num_facilities, student_toolbox, student_generations, student_population_size):
    # Get the reference toolbox and parameters
    reference_toolbox, reference_generations, reference_population_size, reference_tournsize = reference_genetic_algorithm(G, hotspot_nodes, num_facilities)

    # Check generations and population size
    if student_generations == reference_generations:
        print("✅ Number of generations is correct")
    else:
        print(f"❌ Number of generations is incorrect: Expected {reference_generations}, Got {student_generations}")

    if student_population_size == reference_population_size:
        print("✅ Population size is correct")
    else:
        print(f"❌ Population size is incorrect: Expected {reference_population_size}, Got {student_population_size}")

    # Check the evaluate function
    sample_individual = reference_toolbox.individual()
    ref_fitness = reference_toolbox.evaluate(sample_individual)
    student_fitness = student_toolbox.evaluate(sample_individual)
    if ref_fitness == student_fitness:
        print("✅ Evaluate function returns correct output")
    else:
        print(f"❌ Evaluate function returns incorrect output: Expected {ref_fitness}, Got {student_fitness}")

    # Check the tournament size
    student_tournsize = student_toolbox.select.keywords['tournsize'] if 'tournsize' in student_toolbox.select.keywords else None

    if student_tournsize == reference_tournsize:
        print("✅ Tournament size is correct")
    else:
        print(f"❌ Tournament size is incorrect: Expected {reference_tournsize}, Got {student_tournsize}")

# Define the cell magic function
@register_cell_magic
def check_genetic_algorithm_complete(line, cell):
    # Execute the student's code and retrieve relevant variables
    exec(cell, globals())

    # Perform the checks using the global variables (G, hotspot_nodes, num_facilities, etc.)
    perform_checks_genetic_algorithm_complete(G, hotspot_nodes, num_facilities, toolbox, generations, population_size)

# Reference minisum fitness
def ref_calculate_weighted_minisum_distance(G, hotspot_nodes, facility_nodes, weights):
    total_distance = 0

    for hotspot in hotspot_nodes:
        min_distance = float('inf')

        for facility in facility_nodes:
            distance = nx.shortest_path_length(G, source=facility, target=hotspot)
            if distance < min_distance:
                min_distance = distance

        total_distance += min_distance * weights[hotspot]

    return total_distance

def perform_minisum_check():
    if 'calculate_weighted_minisum_distance' in globals():
        ref_max_dist = []
        student_max_dist = []
        count = 0
        for facilities in combinations(G.nodes(), 2):
            ref_max_dist.append(ref_calculate_weighted_minisum_distance(G, hotspot_nodes, facilities, weights))
            student_max_dist.append(calculate_weighted_minisum_distance(G, hotspot_nodes, facilities, weights))
            count += 1
            if count == 10:
                break
        if (ref_max_dist == student_max_dist):
            print("✅ calculate_weighted_minisum_distance is outputting the correct values")
        else:
            print("❌ calculate_weighted_minisum_distance is not outputting the correct values")
    else:
        print("❌ calculate_weighted_minisum_distance is not defined")

@register_cell_magic
def check_minisum(line, cell):
    # Execute the student's code
    exec(cell, globals())
    # Run checks
    perform_minisum_check()

## Simplified Road Networks

The aim of this tutorial is to develop code that finds the optimal facility location on a map, given an identified set of hotspots that most urgently need humanitarian aid. We will get around to working on real maps soon, but for now it will be easier and faster to develop and test our methods on simpler 'fake maps'.

The following code creates a random grid that represents a road network using the Python library NetworkX. Hotspots and facilities are represented by nodes which can be positioned at any point where two roads meet on this network. Run the cell to generate a random network of roads and crisis hotspots.

In [ ]:
# Road Grid Creation
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random

# Parameters
grid_size = (15, 15)  # Size of the grid (rows, columns)
num_hotspots = 3  # Number of hotspots to place

# Create a grid graph
G = nx.grid_2d_graph(grid_size[0], grid_size[1])

# Convert the grid to a graph with only vertical and horizontal connections
# Each node can have up to 4 connections (up, down, left, right)

# Randomly decide which horizontal and vertical edges to keep
def randomize_grid_edges(G, probability=0.55):
    edges_to_remove = [edge for edge in G.edges() if random.random() > probability]
    G.remove_edges_from(edges_to_remove)

# Apply the randomization
randomize_grid_edges(G)

# Get the largest connected component
largest_cc = max(nx.connected_components(G), key=len)
G = G.subgraph(largest_cc).copy()

# Ensure that the number of hotspots does not exceed the number of nodes in the network
nodes = list(G.nodes())
if num_hotspots > len(nodes):
    num_hotspots = len(nodes)

# Randomly place hotspots on the nodes of the graph that are in the network
hotspot_nodes = random.sample(nodes, num_hotspots)

# Prepare for plotting
pos = {node: (node[1], -node[0]) for node in G.nodes()}
edge_labels = {edge: '' for edge in G.edges()}

# Plot the grid network
plt.figure(figsize=(4, 4))
nx.draw(G, pos, with_labels=False, node_size=0, node_color='lightblue', edge_color='gray', alpha=0.6, font_size=10)

# Highlight the hotspots
hotspot_pos = {node: pos[node] for node in hotspot_nodes}
nx.draw_networkx_nodes(G, pos, nodelist=hotspot_nodes, node_color='red', node_size=100, label='hotspots')


# Set plot limits and title
plt.xlim(-1, grid_size[1])
plt.ylim(-grid_size[0], 1)
plt.title('Grid Road Network with Hotspots')
plt.show()


Before we can find an optimal facility configuration, we have to be specific about what 'optimal' means for us in this context - we have to select a measurement of how good a choice of facility locations is.

There are several ways to define that for facility location problems such as this, which we will look at later in the tutorial, but for now let's go with the 'Minimax' rule. Using this rule, we are saying that the optimal facility configuration on the grid minimizes the maximum distance from a crisis hotspot to the nearest humanitarian aid facility.

Try defining a function that takes takes the road network, the set of hotspot nodes and the set of facility nodes as inputs and outputs the maximum distance from a hotspot to the nearest facility.

The NetworkX function,
```nx.single_source_shortest_path_length(Graph, node)``` will be useful - it outputs a Python dictionary where the keys are all the nodes in the graph and the corresponding values are the lengths of the shortest path along the network from the specified starting node to a given end node.



In [ ]:
%%check_calculate_max_distance_to_facility # Leave this here - it will give continuous feedback on your code

num_facilities = 2 # Defining the desired number of facilities as 2
def calculate_max_distance_to_facility(G, hotspot_nodes, facility_nodes):
    # Put your code here:


Now that you have a function that tells you the maximum distance from any hotspot to a facility, it's time to find the configuration of facilities that minimises the function.

The most straightforward method is to find this configuration by brute force, checking all possible configurations and comparing the maximum distance from a crisis hotspot to a facility for each configuration.

Try defining a function in the cell below that returns the best configuration of facilities and the maximum distance from a crisis hotspot to the nearest facility in that case.
```combinations(G.nodes(), num_facilities)```
will return all of the possible choices of 2 nodes from G, the graph representing the road network. It will be very useful for this function.

In [ ]:
%%check_brute_force # Leave this here - it will give continuous feedback on your code

from itertools import combinations
def find_optimal_facilities(G, hotspot_nodes, num_facilities):
    # Put your code here:

    # Return the best facility configuration and the corresponding distance
    return best_facilities, min_max_distance

# Use the function to find the optimal facility locations
optimal_facilities, optimal_max_distance = find_optimal_facilities(G, hotspot_nodes, num_facilities)

Now run the following cell to see the optimal facility locations you just found:

In [ ]:
# Prepare for plotting
pos = {node: (node[1], -node[0]) for node in G.nodes()}

# Ensure positions are set for all nodes
missing_nodes = [node for node in G.nodes() if node not in pos]
if missing_nodes:
    print(f"Missing positions for nodes: {missing_nodes}")

# Plot the results
plt.figure(figsize=(5, 5))
nx.draw(G, pos, with_labels=False, node_size=0, node_color='lightblue', edge_color='gray', alpha=0.6, font_size=10)

# Highlight the optimal facility locations
nx.draw_networkx_nodes(G, pos, nodelist=optimal_facilities, node_color='green', node_size=100, label='Facilities')

# Highlight the hotspots
nx.draw_networkx_nodes(G, pos, nodelist=hotspot_nodes, node_color='red', node_size=100, label='Hotspots')

plt.title('Optimal Facility Locations for hotspots on Grid Road Network')
plt.legend()
plt.show()

You now have solved the optimal facility problem for the random road network and crisis hotspot locations!

Of course, we would now like to apply the method to real maps of cities/countries, but there is a challenge involved in doing so. We have used a brute force method to solve the problem on this network, which was fine because the random graph is at most 15$\times$15 nodes in size. When the number of possible facility locations increases however, the process of finding the optimal solution with this method takes much longer.

To see what I mean, try running the following cell which generates and solves the same problem for a 30$\times$30 random grid. The brute force method on the 15$\times$15 grid should have taken about 10 seconds (give or take - depending on the specifics of the random graph) to find a solution. This time it will take much longer.

In [ ]:
# Parameters
grid_size = (30, 30)  # Size of the grid (rows, columns)
num_hotspots = 3  # Number of hotspots to place

# Create a grid graph
G = nx.grid_2d_graph(grid_size[0], grid_size[1])

# Apply the randomization
randomize_grid_edges(G)

# Get the largest connected component
largest_cc = max(nx.connected_components(G), key=len)
G = G.subgraph(largest_cc).copy()

# Ensure that the number of hotspots does not exceed the number of nodes in the network
nodes = list(G.nodes())
if num_hotspots > len(nodes):
    num_hotspots = len(nodes)

# Randomly place hotspots on the nodes of the graph that are in the network
hotspot_nodes = random.sample(nodes, num_hotspots)

# Use the function to find the optimal facility locations
optimal_facilities, optimal_max_distance = find_optimal_facilities(G, hotspot_nodes, num_facilities)

# Prepare for plotting
pos = {node: (node[1], -node[0]) for node in G.nodes()}

# Ensure positions are set for all nodes
missing_nodes = [node for node in G.nodes() if node not in pos]
if missing_nodes:
    print(f"Missing positions for nodes: {missing_nodes}")

# Plot the results
plt.figure(figsize=(5, 5))
nx.draw(G, pos, with_labels=False, node_size=0, node_color='lightblue', edge_color='gray', alpha=0.6, font_size=10)

# Highlight the optimal facility locations
nx.draw_networkx_nodes(G, pos, nodelist=optimal_facilities, node_color='green', node_size=100, label='Facilities')

# Highlight the hotspots
nx.draw_networkx_nodes(G, pos, nodelist=hotspot_nodes, node_color='red', node_size=100, label='Hotspots')

plt.title('Optimal Facility Locations for hotspots on Grid Road Network')
plt.legend()
plt.show()

# Print the maximum distance from a crisis hotspot to a facility with this solution:
print(f"The maximum distance from a crisis hotspot to the nearest facility is {optimal_max_distance}")

Of course, real maps are much bigger and more invovled than this, so the computing time problem only gets worse when we try to apply the brute force method to real problems.

## Genetic Algorithms

Genetic algorithms offer a solution to this problem. Instead of checking every possible configuration systematically, genetic algorithms check a bunch of configurations (a population), then create a new generation of configurations from the best of the old population with some mutations thrown in for variety and repeats the process, recreating the process of natural selection.

We will use the Python library DEAP to create and run our genetic algorithms. First we have to do some importing - run the following cell to carry that out.

In [ ]:
import random
from deap import base, creator, tools, algorithms

Here's some terminology that will be helpful with navigating the DEAP library and genetic algorithm process:

- Population: All of the configurations currently being considered.

- Generation: A given updated version of the population. Each generation the population is changed by defined methods. The genetic algorithm will run for a number of generations that you decide.

- Individual: Any given configuration in the population is known as an individual.

- Mate: The method you define to get an individual in the next generation by mixing the properties of some of the best individuals in the new generation.

- Mutate: The method by which you introduce some property into an individual in the next generation that isn't taken from an individual in the current generation.

- Fitness: The property of the individuals (or function) that we are trying to maximise. We are trying to minimise a function, so we will simply define the fitness as the negative of our function to minimise and maximise that.

- Tournament: The process we will use for selecting which individuals to use as parents to the next generation. Rather than evaluating the fitness function on the entire poulation, we choose a specified number of random individuals from the population and mate the winners of the tournament.

- Hall of Fame: A list of the individuals with the best fitness across all generations.

The first step in creating our genetic algorithm is defining the functions that return an evaluation of the fitness of individuals, the result of mating two individuals and the result of mutating an individual:

In [ ]:
# Evaluation function for the genetic algorithm
def evaluate(individual):
    facility_nodes = list(set(individual))
    max_distance = calculate_max_distance_to_facility(G, hotspot_nodes, facility_nodes) #We haven't applied the minus sign yet, that will come later
    return (max_distance,)

# Mutation function - it just replaces nodes with random nodes from the network with probability indpb
def mutate(individual, num_nodes, hotspot_nodes, indpb):
    if random.random() < indpb:
        idx = random.randint(0, len(individual) - 1)
        new_node = random.choice(list(G.nodes()))
        while new_node in hotspot_nodes or new_node in individual:
            new_node = random.choice(list(G.nodes()))
        individual[idx] = new_node
    return individual,

# Mating function - it switches each node between the two individuals with a probability of 0.5
def Mate(ind1, ind2):
    size = len(ind1)
    for i in range(size):
        if random.random() < 0.5:
            # Swap genes between the two parents
            if ind1[i] not in ind2:
                ind2[i] = ind1[i]
            if ind2[i] not in ind1:
                ind1[i] = ind2[i]
    return ind1, ind2

Now, we can define the genetic algorithm. DEAP's 'base', 'creator', 'tools' and 'algorithms' modules store the relevant information and functions to use in running the genetic algorithm and run it for us.

In [ ]:
# Genetic Algorithm setup
def genetic_algorithm(G, hotspot_nodes, num_facilities, generations=200, population_size=100):
    num_nodes = len(G.nodes())

    # Create the optimization problem
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))                                                           # This is where the minus sign is applied to our function to minimise
    creator.create("Individual", list, fitness=creator.FitnessMin)                                                        # This defines out individual as a list, whose fitness is given by the above fitness

    toolbox = base.Toolbox()
    toolbox.register("attr_node", lambda: random.choice([node for node in list(G.nodes()) if node not in hotspot_nodes])) # This defines how to get a selection of nodes to make an individual
    toolbox.register("individual", tools.initRepeat, creator.Individual,
                     toolbox.attr_node, n=num_facilities)                                                                 # These lines define what the list that each individual is made of should contain
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)                                            # This defines how to make the population from individuals
    toolbox.register("evaluate", evaluate)                                                                                # This let's the genetic algorithm know about our evaluation function
    toolbox.register("mate", Mate)                                                                                        # This let's the genetic algorithm know about our mating function
    toolbox.register("mutate", mutate, num_nodes=num_nodes, hotspot_nodes=hotspot_nodes, indpb=0.5)                       # This let's the genetic algorithm know about our mutation function
    toolbox.register("select", tools.selTournament, tournsize=5)                                                          # This sets up the generational tournament

    # Create the population
    population = toolbox.population(n=population_size)                                                                    # This creates the population

    # Create the Hall of Fame
    hof = tools.HallOfFame(maxsize=10)

    # Run the genetic algorithm
    algorithms.eaSimple(population, toolbox, halloffame = hof, cxpb=0.5, mutpb=0.2, ngen=generations, verbose=False)      # This runs the eaSimple genetic algorithm process on the population

    # Extract the best individual
    best_individual = tools.selBest(hof, 1)[0]
    best_facilities = list(set(best_individual))
    best_max_distance = evaluate(best_individual)[0]

    return best_facilities, best_max_distance

Now we can run the genetic algorithm and view the result:

In [ ]:
# Run the genetic algorithm
optimal_facilities, optimal_max_distance = genetic_algorithm(G, hotspot_nodes, num_facilities)

# Prepare for plotting
pos = {node: (node[1], -node[0]) for node in G.nodes()}

# Plot the results
plt.figure(figsize=(5, 5))
nx.draw(G, pos, with_labels=False, node_size=0, node_color='lightblue', edge_color='gray', alpha=0.6, font_size=10)

# Highlight the optimal facility locations
nx.draw_networkx_nodes(G, pos, nodelist=optimal_facilities, node_color='green', node_size=100, label='Facilities')

# Highlight the hotspots
nx.draw_networkx_nodes(G, pos, nodelist=hotspot_nodes, node_color='red', node_size=100, label='hotspots')

plt.title('Optimal Facility Locations for hotspots on Grid Road Network')
plt.legend()
plt.show()

# Print the maximum distance from a crisis hotspot to the nearest facility with this solution:
print(f"The maximum distance from a crisis hotspot to the nearest facility is {optimal_max_distance}")

This was certainly faster than the brute force method! But did it give the same results?

It should suggest a configuration of facilities that looks very similar to the brute force method's suggestion, but it is possible that the configurations are not exactly the same. It could be because there are many different configurations that optimise the function we are using to measure success, but it could also be because the method did not manage to find one of these optimal configurations.

This is the trade-off that comes with using methods such as genetic algorithms that take clever shortcuts to reduce the required computation time.

There is no number of generations after which we are guaranteed to have found the optimal configuration, but the proabability of converging to the optimal configuration increases with every additional generation for which we allow the algorithm to run. It may seem like it's worth having to wait a few extra minutes to guarantee you have the correct configuration and you're probably right about that in this case, but the size of the road network is still relatively small. For larger, more realistic road networks the time saving could be a matter of days or even weeks.


## Working with OpenStreetMap

More realistic road networks have been metioned a lot - let's see how we can work with road network from actual maps now. [OpenStreetMap](https://www.openstreetmap.org/#map=3/45.83/-66.62) is an open source map dataset built by a community of voluntary contributors, with roads, railways, walkways, and amenities all included on the map. We can access the dataset with Python using the library OSMnx. RUn the following cell to install it:

In [ ]:
%%capture
!pip install osmnx

This library and dataset is incredibly useful for applying data science and computer simulations to sociogeographical problems, so OSMnx is a worthwhile library to get to know. To fetch a graph of the road network in a given place, use ```graph_from_place(place_name, network_type='drive')```- changing the network type allows you to fetch the walkways, bikeways or combined network of the place.

As with the simple square networks we used above, the network is made up of a graph - a set of nodes connected by a set of edges. We can only place crisis hotspots and facilites on nodes. On a graph retrieved by OSMnx, there only are nodes where roads meet. This is a problem because the optimal facility location isn't neccessarily always at a crossroad or junction. For this reason, the edges in the road network retrieved by OSMnx must be split up into a set of nodes with a certain distance between each pair of nodes.

This is equivalent to creating a 'lattice' in some physics simulations. In both cases, we can't handle an infinite number of choices computationally so we split up the continuous lines into a discrete set of points. By doing this we lose some accuracy, because in the case of our facility location optimisation problem, the optimal position could still be between two nodes. However, we know that the more nodes we use, the more accurate our result will be. We could choose to place nodes every kilometer along the road or every 100 meters along the road - in the first case we will only know the optimal configuration to within a kilometer of each facility location and in the second case we will know it to within 100 meters, but it will take significantly longer to compute a solution in the second case. In this way, we effectively have control over another tradeoff between time saving and accuracy.

The code below imports a map of Manhattan, New York, splits it's roads into nodes positioned 100 meters apart, places three crisis hotspots randomly on the road network and plots it:

In [ ]:
import osmnx as ox
import networkx as nx
import numpy as np
from shapely.geometry import LineString, Point
import matplotlib.pyplot as plt
import random

# Function to fetch and process the road network from OpenStreetMap
def fetch_network(place_name):
    # Fetch the road network from OpenStreetMap
    G = ox.graph_from_place(place_name, network_type='drive')

    # Convert the graph to GeoDataFrames
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

    # Add additional nodes along the edges
    G = add_nodes_along_edges(G, gdf_edges)

    return G

def add_nodes_along_edges(G, gdf_edges, interval=100):
    new_nodes = []
    new_edges = []
    edge_to_new_nodes = {}

    # Store original edge lengths
    edge_lengths = {}
    for u, v, key, data in G.edges(data=True, keys=True):
        if 'length' in data:
            edge_lengths[(u, v, key)] = data['length']

    # Keep track of new nodes and edges
    new_node_id = max(G.nodes()) + 1
    existing_positions = { (data['x'], data['y']): node for node, data in G.nodes(data=True) }

    for idx, row in gdf_edges.iterrows():
        geom = row['geometry']
        u, v, _ = row.name  # Index or identifiers of nodes connected by this edge
        length = edge_lengths.get((u, v, 0), geom.length)  # Default key is 0

        # Generate points along the edge at intervals
        num_points = int(length / interval)

        # If the number of points is less than or equal to 1, skip further processing
        if num_points <= 1:
            continue

        last_node_id = u
        for i in range(1, num_points):
            fraction = i / num_points
            point = geom.interpolate(fraction, normalized=True)
            position = (point.x, point.y)

            # Check if this position already exists
            if position in existing_positions:
                current_node_id = existing_positions[position]
            else:
                # Create a new node
                current_node_id = new_node_id
                new_node_id += 1
                new_nodes.append((current_node_id, position))
                G.add_node(current_node_id, x=point.x, y=point.y)
                existing_positions[position] = current_node_id

            # Add edges from the previous new node to the current new node
            G.add_edge(last_node_id, current_node_id, length=interval)  # Approximate length for this segment
            last_node_id = current_node_id

        # Track the new nodes for the original edge
        edge_to_new_nodes[(u, v)] = (new_nodes[0][0], last_node_id)

    # Remove the old edges after splitting them
    for (u, v), (start_node, end_node) in edge_to_new_nodes.items():
        if G.has_edge(u, v):
            G.remove_edge(u, v)

    return G

def plot_network(G, hotspot_nodes=None, facilities_nodes=None):
    # Define positions for nodes
    pos = {node: (data['x'], data['y']) for node, data in G.nodes(data=True)}

    # Create a figure and axis for plotting
    fig, ax = plt.subplots(figsize=(12, 12))

    # Plot the base network using osmnx
    ox.plot_graph(G, node_size=0, node_color='blue', ax=ax, show=False)

    # Overlay the hotspots
    if hotspot_nodes:
        nx.draw_networkx_nodes(G, pos, nodelist=hotspot_nodes, node_color='red', node_size=100, label='Crisis Hotspots', ax=ax)

    # Overlay the facilities
    if facilities_nodes:
        nx.draw_networkx_nodes(G, pos, nodelist=facilities_nodes, node_color='green', node_size=100, label='Facilities', ax=ax)

    # Add a legend
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, labels)

    plt.title('Road Network with hotspots and Facilities')
    plt.show()

# Fetch the road network
place_name = "Manhattan, USA"
G = fetch_network(place_name).to_undirected()

# Ensure the graph is connected
if not nx.is_connected(G):
    G = G.subgraph(max(nx.connected_components(G), key=len)).copy()

# Example: Randomly select some nodes as hotspots for demonstration
nodes = list(G.nodes())
num_hotspots = 3
hotspot_nodes = random.sample(nodes, min(num_hotspots, len(nodes)))

# Plot the network
plot_network(G, hotspot_nodes=hotspot_nodes)

Now, try defining a new function to run a genetic algorithm using the same mating, mutating and fitness functions from earlier, but now make the population size 200 rather than 100 and define the tournament size to be 10.

In [ ]:
%%check_genetic_algorithm_complete # Leave this here - it will give continuous feedback on your code

def genetic_algorithm(G, hotspot_nodes, num_facilities, generations=50, population_size=200):
    # Start coding here:

    return best_facilities, best_max_distance, toolbox, generations, population_size

# Run the genetic algorithm with two facilities
optimal_facilities, optimal_max_distance, toolbox, generations, population_size = genetic_algorithm(G, hotspot_nodes, 2)

# Plot the network with hotspots and facilities
plot_network(G, hotspot_nodes=hotspot_nodes, facilities_nodes=optimal_facilities)

## Different Fitness Functions

You may have noticed that the optimal configurations by the minimax heuristic for success when you have three crisis hotspots and two facilities is often to have one facility between two of the hotspots and the other fecility near the third hotspot. While this would make it seem like the strategy being used is to cater to two hotspots from one facility and to the third hotspot from the second facility, the optimal configuration in that case would surely be to position the second facility right next-door to the crisis hotspot. However, the 'optimal configurations' given by the minimax heuristic do not always do this, because the fitness function does not change a facility is moved within the region closer to a crisis hotspot than the furthest distance from a hotspot to a facility. Effectively, the minimax heuristic finds the configuration that has the best 'worst case scenario', but doesn't take into account the overall quality of the configuration beyond that.

As was mentioned earlier, there are different ways we could define success in our optimisation problem - using a different measure of success will give us a different optimal configuration. A way to optimise the configuration more holistically is by framing it as a 'minisum' problem.

To define our problem in this way, each facility is given a weight $w_i$ and we aim to minimise $\sum\limits_j\sum\limits_i w_i d_{ij} Y_{ij}$ where $d_{ij}$ is the distance from the $i$th crisis hotspot to the $j$th facility and $Y_{ij}$ is 1 if the $j$th facility is the closest one to the $i$th crisis hotspot and is 0 otherwise. In short, we are minimising a weighted sum of the distances from each hotspot to its nearest facility. In reality, the way we define the weights would depend on the specifics of the problem, but it could for example depend on the number of people affected by the crisis in each hotspot.

The following code defines a dictionary of weights for the random crisis hotspots in Manhattan and shows their values on the map:

In [ ]:
weights = {node : random.randint(1,10) for node in hotspot_nodes}

def plot_network(G, hotspot_nodes=None, facilities_nodes=None):
    # Define positions for nodes
    pos = {node: (data['x'], data['y']) for node, data in G.nodes(data=True)}

    # Create a figure and axis for plotting
    fig, ax = plt.subplots(figsize=(12, 12))

    # Plot the base network using osmnx
    ox.plot_graph(G, node_size=0, node_color='blue', ax=ax, show=False)

    # Overlay the hotspots
    if hotspot_nodes:
        nx.draw_networkx_nodes(G, pos, nodelist=hotspot_nodes, node_color='red', node_size=100, label='Crisis Hotspot', ax=ax)
        for hotspot in hotspot_nodes:
            plt.text(pos[hotspot][0], pos[hotspot][1], f'{weights[hotspot]}', fontsize=14, color='black', ha='center')


    # Overlay the facilities
    if facilities_nodes:
        nx.draw_networkx_nodes(G, pos, nodelist=facilities_nodes, node_color='green', node_size=100, label='Facilities', ax=ax)

    # Add a legend
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, labels)

    plt.title('Road Network with hotspots and Facilities')
    plt.show()

# Plot the network with hotspots and facilities
plot_network(G, hotspot_nodes=hotspot_nodes)

To solve the minisum problem, we need to define a function that outputs the above sum. Give it a try in the cell below:

In [ ]:
%%check_minisum # Leave this here - it will give continuous feedback on your code

def calculate_weighted_minisum_distance(G, hotspot_nodes, facility_nodes, weights):
# Put your code here:


# Evaluation function for the genetic algorithm
def evaluate(individual):
    facility_nodes = list(set(individual))
    max_distance = calculate_weighted_minisum_distance(G, hotspot_nodes, facility_nodes, weights)
    return (max_distance,)

The following cell should now optimise the facility location configuration by minimising this function.

In [ ]:
def genetic_algorithm(G, hotspot_nodes, num_facilities, generations=50, population_size=200):
    num_nodes = len(G.nodes())

    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("attr_node", lambda: random.choice([node for node in G.nodes() if node not in hotspot_nodes]))
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_node, n=num_facilities)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", evaluate)
    toolbox.register("mate", Mate)
    toolbox.register("mutate", mutate, num_nodes=num_nodes, hotspot_nodes=hotspot_nodes, indpb=0.1)
    toolbox.register("select", tools.selTournament, tournsize=10)

    population = toolbox.population(n=population_size)

    # Create the Hall of Fame
    hof = tools.HallOfFame(maxsize=10)

    algorithms.eaSimple(population, toolbox, halloffame = hof, cxpb=0.5, mutpb=0.2, ngen=generations, verbose=False)

    best_individual = tools.selBest(hof, 1)[0]
    best_facilities = list(set(best_individual))
    best_max_distance = evaluate(best_individual)[0]

    return best_facilities, best_max_distance, toolbox, generations, population_size

# Run the genetic algorithm with two facilities
optimal_facilities, optimal_max_distance, toolbox, generations, population_size = genetic_algorithm(G, hotspot_nodes, 2)

# Plot the network with hotspots and facilities
plot_network(G, hotspot_nodes=hotspot_nodes, facilities_nodes=optimal_facilities)

The results from this framing of the problem should look different to when we approached it as a minimax problem - in this way it is clear that in order to get results that are optimal for you, it's important to be careful about how you mathematically describe success.

It might seem to you as if the problems have all had optimal configurations of facilities that you could guess by eye. While you probably could guess fairly good configurations for the three hotspots, two facilities case, when the number of hotspots and facilites increases, things get more difficult and the value of the computational techniques becomes more apparent. Run the following cell to see a setup of crisis hotspots and try to guess what the optimal configuration of four facilities is:

In [ ]:
num_hotspots = 50
hotspot_nodes = random.sample(nodes, min(num_hotspots, len(nodes)))

weights = {node : random.randint(1,10) for node in hotspot_nodes}

# Plot the network with hotspots and facilities
plot_network(G, hotspot_nodes=hotspot_nodes)

Now let's see what the genetic algorithm says:

In [ ]:
# Run the genetic algorithm with two facilities
optimal_facilities, optimal_max_distance, toolbox, generations, population_size = genetic_algorithm(G, hotspot_nodes, 4)

# Plot the network with hotspots and facilities
plot_network(G, hotspot_nodes=hotspot_nodes, facilities_nodes=optimal_facilities)

The ways of framing the facility location optimisation problem that we have looked at here are 'deterministic' - we know the locations of different crisis hotspots and their levels of importance to start with. There are more ways of framing the problem from a deterministic point of view: a maximal covering approach would give each facility a radius within which it can respond and aim to cover as many crisis hotspots as possible within these radii, while a set covering approach would seek to minimise the number of facilities needed to cover all of the crisis hotspots. There are even ways to frame the problem when you want a facility placed as far from social centres as possible without causing to high of a cost - this would be applicable if the facilities involved are waste management sites, for example.

There are also entirely different classes of models for facility location optimisation, where we don't assume to know all of the details to start with (stochastic or robust problems) or we allow the parameters in the problem to change (dynamic problems). These ways of framing the problem are more effective at capturing the reality of the problem but also involve some more advanced techniques.

The paper '[Facility location optimization model for emergency humanitarian logistics](https://www.sciencedirect.com/science/article/pii/S2212420916302576)' describes the models of the problem that we have worked with in this tutorial as well as the dynamic, stochastic and robust facility location problems. If you've enjoyed working through this tutorial, it could be a good place to start finding out more.